## Public Opinion Data Preprocessing 
### Pew Public Opinion Data 2017 - 2021

The goal of this worksheet is to preprocess the data for easy use moving forward. We hope to: 
- remove unnessecary data 
- clean all data so that it has the same labelling conventions (ordinality & missing data point labelling) 
- combine common data across years for analysis 
- export individual datasets and a common one across years

## Methodology 

- Import all .sav files as csv with their original labels. This ensures that ordinal variables are transformed in the same direction 
    - Note: This has typically been produced by using SPSS in the VCL and exporting data with original labels. You also want to grab a dictionary of all the question definitions while you are in there. 
- Squash down all of the categorical varaibles that are labelled with individual country values 
- Create a series of dictionaries to transform the values of the rest of the dataset 
- Drop irrelevant values in each individual dataframe (this will allow a clean dataset to be used for intracountry comparisons) 
- Determine relevant variables to be used across time (this will allow a clean dataset for intercountry comparisons) 
    - This dataset would contain individual responses logged with time (year) with the same column values for the same responses. It would NOT be aggregated so that statistics could be filtered by demongraphic, region, etc before comparisons across time. 
- Export all of this data (1 dataset per year + 1 common dataset) 

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

import ipysheet as ip
from ipysheet import sheet, cell, row, column, cell_range, from_dataframe, to_dataframe 

import warnings
warnings.filterwarnings('ignore')

### Import all of the data with original labelling

In [2]:
years = [2021, 2020, 2019, 2018, 2017]

# grabbing the data across the years data is found 

d = {}

path_raw_file = '/Users/natalie_kraft/Documents/LAS/raw/PewL'
for year in years: 
    d[year] = pd.read_csv(path_raw_file + str(year) + '.csv')
    d[year].columns = d[year].columns.str.lower()

# access dataframe per year through d[year]

In [3]:
# replace Don't Know and Refused with similar verbage 
# this reduces the number of transformations we need to utilize
    
for year in years:   
    d[year] = d[year].replace({"(VOL) Don\'t know" : "Don't know", 
                 "Don\'t know (DO NOT READ)" : "Don't know", 
                 "Dont know (DO NOT READ)" : "Don't know",
                 "(VOL)\xa0Don't know" : "Don't know",
                 "Refused (DO NOT READ)" : "Refused", 
                 "(VOL) Refused" : "Refused", 
                 '(VOL)\xa0Refused' : "Refused", 
                 ' ': "Don't know", 
                 'Don’t know (DO NOT READ)' : "Don't know",
                 "Don\x92t know (DO NOT READ)" : "Don't know",
                 "Refused (DO NOT READ" : "Refused" })
    
# attempted this shortcut, but was affected too high proportion of cells 
#     d[year] = d[year].replace({r"(.*)Don(.*)know(.*)" : "Don't know", 
#             r"(.*)Refused(.*)" : "Refused"}, regex=True)

In [4]:
dOriginal = d

### Reduce dimensionality before preprocessing

The following variables are individually labelled per country: 
- 'd_ptyid_' : party identification 
- 'd_relig_' : religious affiliation 
- 'd_income2_' : wealthy (rich/poor binary var based on cost of living for country) 
- 'd_educ_' : level of education respondant recieved

The country name already provides this affiliation, dimensionality will be reduced to one common variable for each categorical one. The variable names will be transformed such that: 
- 'd_ptyid_' : 'political_affiliation'
- 'd_relig_' : 'religious_affiliation' 
- 'd_income2_' : 'wealthy'
- 'd_educ_' : 'education_level'

In [5]:
def categorical_squash(regex_conven, mapping, new_name, y): 
    
    for year in y: 
        p_temp = d[year].filter(regex=regex_conven).replace(mapping)
        p_temp[new_name] = p_temp.iloc[:, 0]

        for index, name in enumerate(p_temp.columns): 
            p_temp[new_name] = p_temp[new_name].combine_first(p_temp.iloc[:, index])
        
        d[year][new_name] = p_temp[new_name]

In [6]:
# reducing political affiliation
# keep in mind, this only is a label of affiliation, not a favorability toward individual parties 
# favorability exists, but it will not be considered in this round of preprocessing 

categorical_squash('d_ptyid_', {"Don't know" : None}, 'political_affiliation', years)

# reducing religious affiliation 
categorical_squash('d_relig_', {"Don't know" : None}, 'religious_affiliation', years)

# reducing income level
# TODO: This needs updating in accordance with 'd_income_'
categorical_squash('d_income2_', {"Don't know" : None}, 'wealthy', years)
# identifing commonalities across respondant's country
for year in years: 
    d[year]['wealthy'].map(lambda x: 1 if (x is not None) and ('More' in x) else 0).value_counts()
    
categorical_squash('d_educ_', {"Don't know" : None}, 'education_level', years)

__Categorical Squash for Regional Geocoding__ 

In [7]:
# geocoding for 2021, 2020 is region

categorical_squash('region_', {"Don't know" : None}, 'regional_location', [ 2020])

# geocoding for 2019 is region or qs5 

categorical_squash('region_', {"Don't know" : None}, 'temp_r', [2019])
categorical_squash('qs5', {"Don't know" : None}, 'temp_q', [2019])
d[2019]['regional_location'] = d[2019]['temp_r'].combine_first(d[2019]['temp_q'])
d[2019] = d[2019].drop(columns=['temp_r', 'temp_q'])

# geocoding for 2018/2017 is qs5

categorical_squash('qs5', {"Don't know" : None}, 'regional_location', [2018, 2017] )

### Reduce dimensionality before preprocessing
- If the data is a known categorical variable, squash into a common varaible across regions and don't run it through the preprocesser. Or remove categorical variable in its entirity. 
- If the data is not needed, drop it from the dataset 

In [8]:
# Drop unneeded data from the dataset 
# This is a listing of variable names per dataset that is unneeded 

# Absolute column names can be added here
drop = {
    2021 : ['phone_sample','survey', 'weight', 'qdate_e', 'd_adults', 'diversity_goodbad', 'healthsys_reform', 
            'basic_facts', 'public', 'polsys_countryfu', 'climate_behavior', 'usdemocracy_example', 'eu_germanyinfluence'], 
    2020 : ['phone_sample', 'cregion_us', 'density_us', 'covid_change', 'covid_ownfaith', 'covid_countryfaith', 'covid_family',
             'covid_united', 'covid_cooperation', 'pray', 'd_adult_us', 'd_political_scale_us', 'd_ptylean_us',
            'qs8', 'survey', 'weight', 'qdate_e', 'd_born_us', 'compromise'], 
    2019 : ['phone_sample', 'cregion_us', 'density_us', 'fav_hezbollah', 'german_unification', 'germany_standard', 
            'mex_live_us', 'mex_wo_auth', 'survey', 'weight', 'qdate_e', 'd_born_us', 'relparticipate_story', 'fav_muslims_country',  'fav_roma', 'fav_germany', 'receive_money', 'equal_leaders', 'state_us', 
            'influence_finance', 'fav_muslimbulg', 'neighboring_countries', 'eastwest_ger', "influence_relig", 
           'influence_raise', 'econ_integration', 'country_born', 'fav_jews91', "kind_of_marriage", "same_rights", 
           'd_political_scale_us', 'country_national', 'women_rights', 'econ_communism', 'd_political_scale_us', 
           'close_relationship', 'd_adult_us', 'd_ptylean_us', 'nato_def', 'better_gender', 'us_mil_asia'], 
    2018 : ['survey', 'weight', 'qdate_e', 'd_born_us', 'state', 'density', 'usr', 'scregion', 'sstate',
            'susr','sdensity', 'kashmir_military', 'sanc_effrus', 'mex_live_us', 'workauto50yr', 'good_live_us', 
           'receive_money', 'immig_moreless'], 
    2017 : ['survey', 'weight', 'qdate_e', 'd_born_us', 'dem_stable', 'defense_spending', 'desc_day', 
           'dissol_goodbad', 'eu_leavestay', 'euexit_referendum', 'fav_aap', 'fav_india','fav_pak',
            'fav_japan', 'fav_saudi', 'fav_turkey', 'fav_skorea', 'fav_nkorea', 'fav_cuba', 'fav_boko', 'fav_mex',
           'fav_eu', 'fav_germany', 'fav_britain', 'fav_nato', 'swe_join_nato', 'turkey_eu_member', 'dissol_goodbad', 
            'me_role_egypt', 'me_role_saudiarabia', 'me_role_turkey','me_role_iran','me_role_israel',
           'fav_sisi','fav_erdogan','fav_assad','fav_netanyahu','fav_salman','fav_rouhani','fav_abdullahii','refugee_iraqsyr',
            'war_syria_length', 'fav_adtlpolcnty_rousseff', 'fav_adtlpolcnty_luiz', 'fav_adtlpolcnty_temer',
            'fav_lopez', 'fav_radonski', 'fav_allup', 'fav_pri','fav_pan', 'fav_morena', 'fav_prd','fav_modi',
            'fav_kejriwal', 'fav_bjp', 'fav_inc','isr_pal_coexist', 'jewish_settlements', 'd_numcell', 'kashmir_military', 
            'influence_humanrightsorgs', 'nafta_goodbad', 'qsplit',  'racethn', 'racecmb', 'me_role_us', 'prob_kashmir',
           'd_density', 'receive_money', 'concern_country', 'humanrights_motive']
    
}

# to reduce names, all partial (or sets) of columns can be added here
# if the column name contains any part of this value it will be removed 
drop_inc_all = ['survey', 'partyfav', "d_income", "d_race", 'd_ethnicity', "d_ptyid_", "d_educ_", "d_relig_"
               'psu_', 'stratum_', 'american_', 'language', 'pray', "d_working_cell", "abortion", "covid", 'ladder', 
               't.sample', 'homephone', 'confid_johnson', 'confid_macron','confid_merkel', 'confid_castro', 'confid_abe',
               'confid_modi', 'd_hhcell', 'fav_eu', 'fav_un', 'fav_iran', 'fav_nato', 'fav_india', 'fav_japan', 
               'fav_ep', 'fav_ec']

drop_inc = {
    2021 : ['biden_', 'usbest_', 'conflict_', 'climate_intl', 'discrimprob_'], 
    2020 : ['sdlkjafsldjflakjsdlfjl'],
    2019 : ['multiparty', 'churches_', 'language_home', 'ukr_lang', 'brexit_', 'religion20yr', 'id_', 
           'equal_'], 
    2018 : ['qs6', 'qs7', 'qs8', 'qs9', 'qs10', 'qs11', 'cregion', 'robjob4', 'whymove', 
            'fiveyears_', 'indiaus', 'eu_', 'cyberattack_', '20yr', 'planmove', 'modern_educ', 'friends_', 
           'officials_', 'pray_', 'relbehavior', 'pairs_'], 
    2017 : ['brexit_', 'cell_12months', 'church_', 'trump_', 'obama_', 'mfollow_', 'brexit_policy', 'eu_', 'defend_', 
            'smartphone', 'textfreq', 'turkey_', 'maduro', 'nieto', 'mex_', 'gandhi', 'modi', 'putin_', 'duterte', 
           'phil', 'italy_pride', 'stayintouch', 'move', 'friends_', 'pray_', 'd_tenure', 'qs6', 'qs7', 'qs8',
            'qs9', 'qs10', 'qs11', 'nkorea', 'd_relig_practice'], 
}

In [9]:
for year in years: 
    sizeInit = d[year].shape[1]
    
    # ensure that the rest of the data has proper visibility
    for i in d[year].columns:
        try:
            number = d[year][i].value_counts()["Don't know"]
            if number > (d[year].shape[0] * .9): 
                drop[year].append(i)
        except KeyError: 
            # do nothing
            number = 0 
            
    # drop all listed and size-constrained variables 
    d[year] = d[year].drop(columns=drop[year])
    for x in drop_inc[year]: 
        d[year] = d[year].drop([col for col in d[year].columns if x in col], axis=1)
    for x in drop_inc_all: 
        d[year] = d[year].drop([col for col in d[year].columns if x in col], axis=1)

    sizeEnd = d[year].shape[1]
    print("The data from year " + str(year) + " was reduced by " + str(sizeInit - sizeEnd) + " columns.")
    print("The data is now " + str(sizeEnd))

The data from year 2021 was reduced by 226 columns.
The data is now 33
The data from year 2020 was reduced by 192 columns.
The data is now 39
The data from year 2019 was reduced by 521 columns.
The data is now 91
The data from year 2018 was reduced by 482 columns.
The data is now 88
The data from year 2017 was reduced by 783 columns.
The data is now 97


 __We remove all categorical variables in the dataset through dummy variable transformations.__

In [10]:
# listing of all categorical variables to be connected 

found = {
    2021: [],
    2020: [], 
    2019: [], 
    2018: [], 
    2017: []
}

In [11]:
def create_dummy_var(dataset, variable_name, mapping, found): 
    for year in years: 
        if variable_name in dataset[year].columns: 
            print(variable_name + ' variable found in ' + str(year))
            dummy_demo = pd.get_dummies(dataset[year][variable_name].map(mapping))
            found[year].extend(list(dummy_demo.columns))
            
            # need to merge dummy variables into df 
            dataset[year] = pd.concat([dataset[year], dummy_demo], axis=1)
            dataset[year] = dataset[year].drop(columns=[variable_name])

In [12]:
# some variables are categorical 
# they will be transformed into dummy variables and their original label will be removed 

# econ_power
econ_power_mapping = {
    "The United States": "US_econ_power", 
    "China": "China_econ_power",
    "Japan": "Japan_econ_power",
    "The countries of the European Union": "EU_econ_power",
    "(VOL) None / There is no leading economic power": "no_econ_power",
    "None / There is no leading economic power (DO NOT READ)" : "no_econ_power",
    "(VOL) Other": "other_econ_power", 
    "Other (DO NOT READ)" : "other_econ_power"
}

# maps onto dummy variables
create_dummy_var(d, 'econ_power', econ_power_mapping, found)

econ_power variable found in 2020
econ_power variable found in 2019
econ_power variable found in 2018
econ_power variable found in 2017


In [13]:
# 'us_or_china'
econ_us_china = {
    "The United States" : "prefer_us_econ", 
    "China" : "prefer_china_econ", 
    "Economic ties to both countries are equally important (DO NOT READ)" : "both_china_econ"
}

create_dummy_var(d, 'us_or_china', econ_us_china, found)
for year in years: 
    if 'both_china_econ' in d[year].columns: 
        d[year]['prefer_us_econ'] = d[year]['both_china_econ'] + d[year]['prefer_us_econ']
        d[year]['prefer_china_econ'] = d[year]['both_china_econ'] + d[year]['prefer_china_econ']
        d[year].drop(columns=['both_china_econ', 'no_econ_power', 'other_econ_power'])

us_or_china variable found in 2021
us_or_china variable found in 2019


In [14]:
# econ_power
world_leader_mapping = {
    "The U.S. is the world’s leading power": "US_better_worldleader", 
    "China is the world’s leading power": "China_better_worldleader",
    "Both (DO NOT READ)": "both_better_worldleader"
}

# maps onto dummy variables
create_dummy_var(d, 'worldleader_uschina', world_leader_mapping, found)
for year in years: 
    if 'both_better_worldleader' in d[year].columns: 
        d[year]['US_better_worldleader'] = d[year]['both_better_worldleader'] + d[year]['US_better_worldleader']
        d[year]['China_better_worldleader'] = d[year]['both_better_worldleader'] + d[year]['China_better_worldleader']
        d[year].drop(columns='both_better_worldleader')

worldleader_uschina variable found in 2018


### Identify mapping for transformations
- Create a list for variables where no transformations are needed 
- Create all mappings for variables
- Search through all variables and map those with similar corresponding labels 

In [15]:
# These column values don't need to be transformed, but we do want to keep them in the dataset 
# They are either discrete values or they are regional/naming conventions. 


keep = ['id', 'country', 'sex', 'age', "d_density", 'd_hhpeople', 'political_scale2', 'qdate_s',  "d_density", 
        'muslim_branch', 'political_affiliation', 'religious_affiliation', 'education_level', 'wealthy', 'regional_location']

keep_inc = {
    2021 : [ 'id'], 
    2020 : [ 'state_us'],
    2019 : [ "allies_new_1", "threats_new_1", 'us_or_china'], 
    2018 : [ 'qlang', "d_adult_us", ], 
    2017 : [ 'qlang', "d_adult_us",], 
}


In [16]:
missing_vars = {
    "Don't know": 8, 
    "Refused" : 9
}

sat_bin = {
    "Dissatisfied": 0, 
    "Satisfied": 1, 
}

sat_q = {
    "Not too satisfied": 2, 
    "Not at all satisfied" : 1, 
    "Somewhat satisfied": 3, 
    "Very satisfied": 4, 
}

good_bad_q = {
    "Somewhat good" : 3,
    "Somewhat bad" : 2,
    "Very bad" : 1,
    "Very good" : 4
}

better_t = {
    "Worse off" : 1, 
    "Gotten worse" : 1,
    "Worse" : 1,
    "Better off" : 3, 
    "Better" : 3, 
    "Gotten better" : 3, 
    "Same (DO NOT READ)" : 2, 
    "Stayed about the same" : 2, 
    "About the same" : 2
}

prob_q = {
    "Very big problem" : 4, 
    "Moderately big problem" : 3, 
    "Small problem" : 2, 
    "Not a problem at all" : 1, 
}

fav_q = {
    "Somewhat favorable" : 3, 
    "Somewhat unfavorable" : 2, 
    "Very favorable": 4, 
    "Very unfavorable" : 1
}

amount_q = {
    "Great deal" : 4, 
    "A great deal" : 4,
    "Fair amount" : 3, 
    "A fair amount" : 3,
    "Not too much" : 2, 
    "Not at all" : 1
}

approval_q = {
    "Approve": 3, 
    "Strongly approve": 4, 
    "Disapprove" : 2, 
    "Strongly disapprove" : 1
} 

confid_q = {
    "No confidence at all" : 1, 
    "Not too much confidence" : 2, 
    "Some confidence" : 3, 
    "A lot of confidence" : 4
}

right_t = {
    "About right" : 2, 
    "Too great" : 3, 
    "Too small" : 1
}

yesno_bin = {
    "No" : 0, 
    "Yes" : 1
}

influe_q = {
    "Great deal of influence" : 4,
    "Very good influence" : 4, 
    "Fair amount of influence" : 3, 
    "Good influence" : 3, 
    "Bad influence" : 2, 
    "Very bad influence" : 1, 
    "Not too much influence" : 2, 
    "No influence at all" : 1, 
    "No influence (DO NOT READ)" : 8
}
  
mil_bin = {
    "Yes, would use military force" : 1, 
    "Yes, should use military force" : 1, 
    "No, would not use military force" : 0, 
    "No, should not use military force" : 0
}

import_q = {
    "Very important" : 4,  
    "Somewhat important" : 3,
    "Not very important" : 2,
    "Not too important" :2, 
    "Not at all important": 1, 
    "Not important at all": 1  
}

roles_t = {
    "More important role" : 3, 
    "Doing more" : 3, 
    "Less important role" : 1, 
    "Doing less" : 1, 
    "As important as 10 years ago" : 2, 
    "U.S. does not help (DO NOT READ)" : 2, 
    "About the same" : 2
}

threat_t = {
    "Major threat" : 3, 
    "Not a threat" : 1, 
    "Minor threat" : 2, 
    'Very concerned' : 3,
    'Somewhat concerned' : 2, 
    'Not too concerned' : 2, 
    'Not at all concerned' : 1
}

changes_t = {
    'It needs to be completely reformed' : 4, 
    'It needs major changes' : 3, 
    'It needs minor changes' : 2, 
    'It doesn’t need to be changed' : 1
}

god_bin = {
    "It is necessary to believe in God in order to be moral and have good values" : 1, 
    "It is not necessary to believe in God in order to be moral and have good values" : 0
}

china_bin = {
    "The U.S. should try to promote human rights in China, even if it harms economic relations with China" : 1,
    '(response in COUNTRY) should try to promote human rights in China, even if it harms econo' : 1, 
    "The U.S. should prioritize strengthening economic relations with China, even if it means not addressing human rights iss" : 0,
    '(response in COUNTRY) should prioritize strengthening economic relations with China, even if it means not addressing' : 0
}

priority_q = {
    "Top priority" : 4, 
    "Important but lower priority" : 3, 
    "Not too important" : 2, 
    "Should not work on this issue" : 1
}

trust_bin = {
    "In general, most people can be trusted" : 1,
    "In general, most people cannot be trusted" :0
}

econ_q = {
    "Improve a lot" : 4, 
    "Improve a little" : 3,
    "Worsen a little" : 2,
    "Worsen a lot" :1, 
    "Remain the same" : 2.5
}

trust_q = {
    "A lot" : 4, 
    "Somewhat" : 3, 
    "Not much" : 2, 
    "Not at all" : 1
}

enemy_t = {
    "Competitor" : 2,     
    "Enemy" :3,
    "Partner" :1
}

agree_q = {
    "Mostly disagree" : 2, 
    "Completely disagree" :1, 
    "Mostly agree" : 3, 
    "Completely agree" : 4, 
}

goodbad_b = {
    "Bad thing" : 0, 
    "Good thing" : 1, 
    "Neither good nor bad" : 0, 
    "Neither (DO NOT READ)" : 0,
    "Both (DO NOT READ)" :0
}

goodbad_b2 = {
    "Investment from China is a good thing" : 1, 
    "Investment from China is a bad thing" : 0
}

posneg_b  = {
    "Positive" : 1, 
    "Negative" : 0, 
    "Neither/both (DO NOT READ)" : 0   
}

opto_b = {
    "Optimistic" : 1, 
    "Pessimistic" : 0, 
    "Neither (DO NOT READ)" : 0
}

smart_b = {
    "Smartphone" : 1,
    "Not a smartphone" : 0
}

cell_b = {
    "Yes, someone in household has cell phone" : 1, 
    "No" : 0 
}

global_b = {
    "should act as part of a global community that works together to solve problems" : 1,                      
    "should act as independent nations that compete with other countries and pursue their own interests" : 0, 
    "Both (DO NOT READ)" : 1, 
    "Neither (DO NOT READ)" : 0                  
}

homo_b = {
    "Homosexuality should be accepted by society" : 1,
    "Homosexuality should not be accepted by society" : 0,  
}

relat_b = {
    "Building a strong relationship with China on economic issues" : 0,
    "Getting tougher with China on economic issues" : 1
}

news_q = {
    "Very well" : 4, 
    "Somewhat well" :3, 
    "Not too well" : 2, 
    "Not well at all" : 1, 
    "News organizations should not do this (DO NOT READ)" : 0
}

news_b = {
    "It is never acceptable for a news organization to favor one political party over others when reporting news" : 0, 
    "It is sometimes acceptable for a news organization to favor one political party over others when reporting news" : 1
}

respect_b = {
    "Yes, respects personal freedoms" : 1, 
    "No, does not respect personal freedoms" : 0 
}

support_b = {
    "Support" : 1, 
    "Oppose" : 0
}
    
civic_q = {
    "Have done in the past year" : 4, 
    "Have done in the more distant past" : 3, 
    "Have not done, but might do" : 2, 
    "Have not done and would never, under any circumstances, do" : 1
}

increase_t = {
    "Increase" : 3, 
    "Decrease" : 1,
    "Does not make a difference" : 2
}

nukes_t = {
    "Too much" : 3, 
    "About what needs to be done OR" : 2, 
    "Too little" : 1
}

jobs_t = {
    "Job creation" : 3, 
    "Job losses" : 1, 
    "Does not make a difference" : 2
}

likely_q = {
    "Very likely" : 4, 
    "Somewhat likely" : 3, 
    "Not too likely" : 2, 
    "Not at all likely" : 1
}

social_s = {
    "Several times a day" : 7,  
    "Once a day" : 6,       
    "Several times a week" : 5, 
    "Once a week" : 4, 
    "Several times a month" : 3, 
    "Once a month" : 2, 
    "Less than once a month" : 1,  
    "Never" : 0,                    
}

better_place_t = {
    "A better place to live" : 3, 
    "A worse place to live" : 1, 
    "Doesn\'t make much difference either way (DO NOT READ)" : 2
}

reliability = {
    "Very reliable" : 4,
    "Somewhat reliable" : 3, 
    "Not too reliable" : 2,
    "Not at all reliable" : 1, 
}

dictionaries = [sat_bin, good_bad_q, sat_q, better_t, fav_q, amount_q, approval_q, confid_q, right_t, 
               yesno_bin, influe_q, mil_bin, import_q, roles_t, threat_t, prob_q, trust_q]

dictionaries_niche = [god_bin, trust_bin, china_bin, econ_q, enemy_t, agree_q, goodbad_b, goodbad_b2,  
                      posneg_b, opto_b, smart_b, global_b, homo_b, relat_b, news_q, respect_b, support_b, 
                     civic_q, increase_t, nukes_t, jobs_t, likely_q, social_s, priority_q, news_b, 
                     cell_b, better_place_t, reliability, changes_t]

# ensures the missing variables are included in the datasets 

for i in dictionaries: 
    i.update(missing_vars)
    
for i in dictionaries_niche: 
    i.update(missing_vars)

In [17]:
# This function provides a matching mechanism for data labels into a numeric scale 
# This scale is constant across years (where positive responses are ranked highest)
# The original dataset is overrridden with these transformations 
def preprocess(year, dictionaries, found):
    
    for i in d[year].columns: 
        if ('qs' not in i) and ('region' not in i) and (i not in keep) and (i not in keep_inc[year]) and (i not in found[year]): 
            for di in dictionaries: 
                if len(set(d[year][i]).difference(set(di.keys()))) == 0: 
                    found[year].append(i)
                    d[year][i] = d[year][i].map(di)
                    break
                       
            for di in dictionaries_niche: 
                if len(set(d[year][i]).difference(set(di.keys()))) == 0: 
                    found[year].append(i)
                    d[year][i] = d[year][i].map(di)
                    break
                       
        else: 
            found[year].append(i)
                
    return d[year], found[year]

In [18]:
# NOTE 2021 columns "reliable_us" --> "relations_us" and "climate_concern" --> "intthreat_climatechange"

In [19]:
for year in years: 
    
    print("Currently, we are preprocessing year " + str(year))
    d[year], found[year] = preprocess(year, dictionaries, found)
    print("There were " + str(len(found[year])) + " columns preprocessed.")
    print("This means that there were " + str(len(set(d[year].columns).difference(set(found[year])))) + " columns left to support: ")
    print(set(d[year].columns).difference(set(found[year])))
    print("")
    print("--------------------------------------------")

Currently, we are preprocessing year 2021
There were 36 columns preprocessed.
This means that there were 0 columns left to support: 
set()

--------------------------------------------
Currently, we are preprocessing year 2020
There were 48 columns preprocessed.
This means that there were 0 columns left to support: 
set()

--------------------------------------------
Currently, we are preprocessing year 2019
There were 107 columns preprocessed.
This means that there were 0 columns left to support: 
set()

--------------------------------------------
Currently, we are preprocessing year 2018
There were 100 columns preprocessed.
This means that there were 0 columns left to support: 
set()

--------------------------------------------
Currently, we are preprocessing year 2017
There were 108 columns preprocessed.
This means that there were 0 columns left to support: 
set()

--------------------------------------------


## Data Merge - reducing dimensionality 

- each year has a time frame added when the survey was conducted 
- data is merged on like column names 
- dimensionality reduction showcased 

In [20]:
# adding the survey year to the dataframe 
for year in years: 
    d[year]['surveyYear'] = year

In [21]:
# assessing data dimensionality 
count = 0
col = 0
for year in years: 
    count = d[year].shape[0] + count
    col = d[year].shape[1] + col
    print("The dimensionality of this year is " + str(count) + " by " + str(col))

The dimensionality of this year is 16254 by 35
The dimensionality of this year is 30530 by 79
The dimensionality of this year is 68956 by 178
The dimensionality of this year is 99065 by 272
The dimensionality of this year is 141018 by 375


In [22]:
# data merge 

df = pd.DataFrame()
for year in years: 
    df = df.append(d[year].reset_index())
    
print("The dimensionality of the combined data is " + str(df.shape[0]) + " by " + str(df.shape[1]))
print("In total, we have captured " + str(round(count * 100 / df.shape[0], 2)) + "% of the data after the merge.")
print("There has been a " + str(round((col - df.shape[1]) * 100 / df.shape[1], 2)) + "% decrease in column through overlapping.")

The dimensionality of the combined data is 141018 by 244
In total, we have captured 100.0% of the data after the merge.
There has been a 53.69% decrease in column through overlapping.


In [23]:
print("Across all of our years of data " + str(df.dropna(axis=1).shape[1]) + " columns are present across the dataset. ")

Across all of our years of data 10 columns are present across the dataset. 


## Identify Commonalities 

Through individual parsing of data, it is possible that variables denoting the same question with different variable names have been listed as seperate columns in the merged dataset. We look to identify any variables of identical questions that were NOT merged accordingly and manually adjust the final spreadsheet. 

Currently, we have a secondary sheet logged within each original file which contains the actual questioned asked of respondants. Our goal is to identify similarity between these questions, we then can confirm these mappings and then transform the overlapping variables to the same name. Here is the methodology: 

- Create a mapping between the question variable name and the question itself for each variable in every year. 
- Use WordMoverDistance to identify semantic similarities 
- Provide a listing of variables with the questions listed for approval 
- Verified pairs will be listed in a dataframe 
- All verified pairs will be replaced with a common variable name 

Then we can return to merging our data together. 

__View a listing of all of the variables for each year and how they match across years with__ 

``` 
varAll
```

In [ ]:
# add all common variable names to a separate dataframe 
# columns - years & final_var_name
# rows - variable names 
# every row will be given a final_var_name which will replace the var_name for that year before the merge 

# create a dataframe showing which variables map to which final_variable_names 
# first instantiate it with the variables which do match across df 

varAll = {
    2020 : pd.DataFrame(d[2020].columns, columns=['2020']).reset_index(drop=True), 
    2019 : pd.DataFrame(d[2019].columns, columns=['2019']).reset_index(drop=True), 
    2018 : pd.DataFrame(d[2018].columns, columns=['2018']).reset_index(drop=True), 
    2017 : pd.DataFrame(d[2017].columns, columns=['2017']).reset_index(drop=True)
}

var = varAll[2020]
var['temp'] = var['2020']
var = var[["temp", "2020"]]


# for every year 
for year in years[1:]: 
    # merge in the next year 
    var = var.merge(varAll[year], left_on = 'temp', right_on = str(year), how='outer')
    var = var.fillna('nan')
    # add in the final_var_name to ensure that all variables are being matched 
    var['temp'] = var['temp'].combine(var[str(year)], func = (lambda x1, x2: x1 if x1 is not 'nan' else x2))
    
varOriginal = var 

In [ ]:
# making an interactive table for ease of mapping variables 
# This was deemed computationally too expensive, instead the table is exported for editting and will be reimported 
# when complete 
# sheet = from_dataframe(var)
# sheet

__Find the questions for each variable name through searching__

```
qMap[year][variable_name]
```

In [ ]:
# import the question labels in 

# remove the 'Q#' label at the beginning 

# pair the question labels with the variable names 
# we are building a dictionary (accessible by year : [(label, name)] pairs) 

qMap = {
    2020 : {}, 
    2019 : {}, 
    2018 : {}, 
    2017 : {}
}

for year in years: 
    tempdf = pd.read_excel("/Users/natalie_kraft/Documents/LAS/bridash/data_processing/pew/pewQVDict.xlsx", sheet_name = str(year))
    # drop any variables which are not used by the main source code, there is no need to further process these 
    tempdf['variable_name'] = tempdf['variable_name'].str.lower()
    tempdf[tempdf['variable_name'].isin(list(d[year].columns)) == True].shape
    qMap[year] = dict(zip(tempdf['variable_name'], tempdf['question']))
    
    # remove all of the 'Q'
    for key in qMap[year]: 
        if qMap[year][key][:1] == 'Q': 
            qMap[year][key] = qMap[year][key].split(".")[1]

In [ ]:
# remove all columns from key:value pair that are no longer in dataset 
for year in years: 
    pop_variables = list(set(qMap[year].keys()).difference(set(varAll[year][str(year)])))
    [qMap[year].pop(x, None) for x in pop_variables]

__Identify semantic similarities in question labels__

In [ ]:
# utilize Word2Vec to identify semantic similarities in question labels 
# print out similar questions and variable names for clarification 



In [ ]:
# TODO: identify where we will collect the "correct" pairings for reference and to reduce the list. 
# If we can identify a threshold, that could work to automatically add them to the final df 
# *** We can manually add values by establishing an interactive table in the notebook 

## Tranformations

For consistency, conduct your final transformations prior to export. This includes mapping all religious affiliations to categories. 

Note: consider bridging over political affiliations as well. Requires lots of research on political groups. 

In [ ]:
# transformation for all religious beliefs. 
# If the label CONTAINS the key, it should be grouped into the value categories. 
relig_transform  = {
    r"(.*)Christian(.*)" : "Christian", 
    r"(.*)Unitarian(.*)" : "Christian", 
    r"(.*)Agnostic(.*)" : "Agnostic", 
    r"(.*)African(.*)" : "traditional African religion", 
    r"(.*)Atheist(.*)" : "Atheist", 
    r"(.*)Baha(.*)" : "Bahai", 
    r"(.*)Buddhis(.*)" : "Buddhist",
    r"(.*)Buddist(.*)" : "Buddhist", 
    r"(.*)Catholic(.*)" : "Catholic", 
    r"(.*)Confucianism(.*)" : "Confucianism", 
    r"(.*)Congregationalist(.*)" : "Protestant", 
    r"(.*)Druze(.*)" : "Druze", 
    r"(.*)Evangelical(.*)" : "Protestant", 
    r"(.*)Hindu(.*)" : "Hindu", 
    r"(.*)Iglesia ni Cristo(.*)" : "Christian", 
    r"(.*)Indigenous religion(.*)" : "Indigenous religion", 
    r"(.*)Jain(.*)" : "Jain", 
    r"(.*)Jehova(.*)" : "Restorationist Christian",  
    r"(.*)Jew(.*)" : "Jewish", 
    r"(.*)Lutheran(.*)" : "Protestant", 
    r"(.*)Mormon(.*)" : "Restorationist Christian", 
    r"(.*)Muslim(.*)" : "Muslim", 
    r"(.*)No(.*)" : None, 
    r"(.*)Orthodox(.*)" : "Catholic", 
    r"(.*)Pentecostal(.*)" : "Protestant", 
    r"(.*)Presbyterian(.*)" : "Protestant", 
    r"(.*)Protestant(.*)" : "Protestant", 
    r"(.*)Sikh(.*)" : "Sikh", 
    r"(.*)Something else(.*)" : "religious",
    r"(.*)Spiritist(.*)" : "Spiritist", 
    r"(.*)Refused(.*)" : None,
    r"(.*)Afrobrazilian religion(.*)" : "Afrobrazilian religion", 
    r"(.*)Unification(.*)" : "Christian", 
    r"(.*)Unitarian(.*)" : "Christian", 
    r"(.*)Yes(.*)" : "religious", 
}

df['religious_affiliation'] = df['religious_affiliation'].replace(regex=relig_transform)

## Appendix 

Throughout the preprocessing of this data, several areas of expansion were identified. These include: 
- __parsing of political affililation__ - data currently includes favorability to 'mainstream' parties and party affiliation. Transformation to leaning across countries could be valuable. Currently, all political identification moved toward one generic variable "political_party". 
- __updating income level__ - there is a variable 'd_income2_' that supposedly categorizes wealth. This label is inaccurate and doesn't log all of respondants wealth, regardless of a variable 'd_income_' that has this granularity. 

In [ ]:
df.columns

In [ ]:
list(df.columns)

In [ ]:
df[0:50]

In [24]:
{ "id":"Dalian", "title":"Dalian Container Terminal Co., Ltd.", "share": "19", "target": "18", "TEU": "9,500,000", "depth": "17.8", "url": "http://www.dct.com.cn/"}, { "id":"Dalian", "title":"Dalian Dagang China Shipping Container Terminal Co., Ltd.", "share": "35", "target": "1", "TEU": "100,000", "depth": "9.1", "url": ""}, { "id":"Dalian", "title":"Dalian Automobile Terminal Co., Ltd.", "share": "24", "target_v": "3", "TEU_v": "780,000", "depth": "11.0", "url": ""}, { "id":"Tianjin", "title":"Tianjin Port Container Terminal Co., Ltd.", "share": "51", "target": "13", "TEU": "6,000,000", "depth": "12.0-17.0", "url": "http://www.tianjin-port.com/port/Show.asp?m=1&d=1096"}, { "id":"Yingkou", "title":"Yingkou Container Terminals Company Limited", "share": "50", "target": "2", "TEU": " 1,200,000 ", "depth": "14.0", "url": "http://221.202.72.96:9090/"}, { "id":"Yingkou", "title":"Yingkou New Century Container Terminal Co., Ltd.", "share": "40", "target": "2", "TEU": "1,200,000", "depth": "15.5", "url": ""}, { "id":"Jinzhou", "title":"Jinzhou New Age Container Terminal Co., Ltd.", "share": "51", "target": "2", "TEU": "800,000", "depth": "15.4", "url": "http://www.jnct.com.cn/"}, { "id":"Qinhuangdao", "title":"Qinhuangdao Port New Harbour Container Terminal Co., Ltd.", "share": "30", "target": "2", "TEU": "950,000", "depth": "15.8", "url": "http://bid.coal.com.cn/Member/Company_316070.htm"}, { "id":"Qingdao", "title":"Qingdao Port International Co., Ltd.", "share": "19.79", "target": "24", "TEU": "10,000,000", "target2": "62", "TEU2": "207,020,000", "depth": "", "url": "http://www.qingdao-port.com"}, { "id":"Qingdao", "title":"Qingdao Port Dongjiakou Ore Terminal Co., Ltd.", "share2": "25", "target2": "3", "TEU2": "29,000,000", "depth2": "20.0-25.0", "url": ""}, { "id":"Shanghai", "title":"Shanghai Pudong International Container Terminals Limited", "share": "30", "target": "3", "TEU": "2,300,000", "depth": "12.0", "url": "http://www.spict.com/"}, { "id":"Shanghai", "title":"Shanghai Mingdong Container Terminals Limited", "share": "20", "target": "7", "TEU": "5,600,000", "depth": "12.8", "url": "https://www.smct.com.cn"}, { "id":"Ningbo", "title":"Ningbo Yuan Dong Terminals Limited", "share": "20", "target": "3", "TEU": "3,000,000", "depth": "17.1", "url": ""}, { "id":"Ningbo", "title":"Ningbo Meishan Bonded Port New Harbour Terminal Operating Co., Ltd.", "share": "20", "target": "2", "TEU": "1,200,000", "depth": "15.6", "url": ""}, { "id":"Lianyungang", "title":"Lianyungang New Oriental International Terminals Co., Ltd.", "share": "55", "target": "4", "TEU": "1,400,000", "depth": "11.5-15.0", "url": "http://www.lnoct.com"}, { "id":"Taicang", "title":"Taicang International Container Terminal Co., Ltd.", "share": "39.04", "target": "2", "TEU": "550,000", "depth": "12.0", "target2": "2", "TEU2": "4,000,000", "depth2": "12.0", "url": "http://www.taicangterminals.com/"}, { "id":"Wuhan", "title":"Wushan CSP Terminal Co., Ltd.", "share": "70", "target": "4", "TEU": "77,200", "depth": "6.4", "url": "", "target2": "4", "TEU2": "4,200,000","depth2": "6.4"}, { "id":"Nantong", "title":"Nantong Tonghai Port Co., Ltd.", "share": "51", "target": "3", "TEU": "1,470,000", "depth": "9.0-11.0", "target2": "1", "TEU2": "5,370,000", "depth2": "6.0"/*, "remark":""*/}, { "id":"Xiamen", "title":"Xiamen Ocean Gate Container Terminal Co., Ltd.", "share": "70", "target": "4", "TEU": "2,600,000", "depth": "15.0", "target2": "1", "TEU2": "4,000,000", "depth2": "6.6-13.6", "url": "http://www.coscoyh.com.cn/"}, { "id":"Quanzhou", "title":"Quan Zhou Pacific Container Terminal Co., Ltd.", "share": "82.35", "target": "5", "TEU": "3,000,000", "depth": "11.6-15.1", "target2": "2", "TEU2": "1,000,000", "depth2": "5.1-9.6", "url": "http://www.qpct.com.cn/"}, { "id":"Jinjiang", "title":"Jinjiang Pacific Ports Development Co., Ltd.", "share": "80", "target": "2", "TEU": "600,000", "depth": "9.5-15.3", "target2": "2", "TEU2": "4,200,000", "depth2": "7.5-9.5", "url": "http://jppdc.com.cn/lxweb/index.asp"}, { "id":"Kaohsiung", "title":"Kao Ming Container Terminal Corp.", "share": "20", "target": "4", "TEU": "2,800,000", "depth": "16.5", "url": "http://www.kmct.com.tw/index.php"}, { "id":"Shenzhen", "title":"Yantian International Container Terminals Co., Ltd.", "share": "14.59"/*, "target": "5", "TEU": "4,500,000", "depth": "14.0-15.5", "url": ""*/, "bf1":"<b>Phase 1, 2</b>"}, { "id":"Shenzhen", "title":"", "share": "13.36", /*"target": "20", "TEU": "13,000,000", "depth": "14.0-17.6", "url": "https://www.yict.com.cn/index.html?locale=zh_CN",*/ "bf1":"<b>Phase 3</b>"}, { "id":"Shenzhen", "title":"", "target": "20", "TEU": "13,000,000", "depth": "14.0-17.6", /*"url": "https://www.yict.com.cn/index.html?locale=zh_CN",*/ "bf1":"<b>Phase 1, 2, 3</b>"}, { "id":"Guangzhou", "title":"Guangzhou South China Oceangate Container Terminal Company Limited", "share": "39", "target": "6", "TEU": "4,200,000", "depth": "15.5", "url": "https://www.goct.com.cn/"}, { "id":"Guangzhou", "title":"Nansha Stevedoring Corporation Limited of Port of Guangzhou", "share": "40", "target": "4", "TEU": "5,000,000", "depth": "14.5-15.5","url":"http://www.gnict.com/"}, { "id":"Hong", "title":"COSCO-HIT Terminals (Hong Kong) Limited", "share": "50", "target": "2", "TEU": "1,800,000", "depth": "15.5", "url": "https://hutchisonports.com/ports/world/cosco-hit-terminals-hong-kong-limited-cht/"}, { "id":"Hong", "title":"Asia Container Terminals Limited", "share": "60", "target": "2", "TEU": "1,600,000", "depth": "15.5", "url": "https://hutchisonports.com/ports/world/asia-container-terminals-limited-act/"}, { "id":"Qinzhou", "title":"Guangxi Beibu Gulf International Container Terminal Co., Ltd.", "share": "30.69", "target": "6", "TEU": "3,600,000", "depth": "15.1"}, { "id":"Qinzhou", "title":"Beibu Gulf Port Co., Ltd.<sup>Note</sup>", "share": "10.65", "target": "18.0", "TEU": "10,800,000", "target2": "100", "TEU2": "268,400,000", "depth": "", "remark":"Note: The target no. of berths and the target designed annual handing capacity do not include Beibu Gulf Terminal.", "url": "https://www.bbwport.cn"}, { "id":"Piraeus", "title":"Piraeus Container Terminal Single Member S.A.", "share": "100", "target": "8", "TEU": "6,200,000", "depth": "14.5-19.5", "url": "http://www.pct.com.gr/"}, { "id":"Zeebrugge", "title":"CSP Zeebrugge Terminals NV", "share": "90", "target": "3", "TEU": "1,300,000", "depth": "17.5", "url": "http://www.cspterminals.be/"}, { "id":"Noatum", "title":"CSP Iberian Valencia Terminal, S.A.U.", "share": "51", "target": "6", "TEU": "4,100,000", "depth": "16", "url": "https://www.cspspain.com/EN"}, { "id":"Noatum", "title":"CSP Iberian Bilbao Terminal, S.L.", "share": "39.51", "target": "3", "TEU": "1,000,000", "depth": "21"}, { "id":"COSCO", "title":"COSCO-PSA Terminal Private Limited", "share": "49", "target": "5", "TEU": "4,850,000", "depth": "18.0", "url": "https://www.singaporepsa.com/our-business/terminals/joint-venture-terminals"}, { "id":"Vado", "title":"Reefer Terminal S.p.A.", "share": "40", "target": "2", "TEU": "250,000", "depth": "14.5", "target2_p": "2", "TEU2_p": "600,000", "depth2": "14.1"}, { "id":"Vado", "title":"Vado Container Terminal", "share": "40", "target": "2", "TEU": "860,000", "depth": "17.25", "url": "http://www.apmterminals.com/en/operations/europe/vado-ligure"}, { "id":"Kumport", "title":"Kumport Liman Hizmetleri ve Lojistik Sanayi ve Ticaret <nobr>A. Ş.</nobr>", "share": "26", "target": "6", "TEU": "2,100,000", "depth": "15.0-16.5", "url": "http://www.kumport.com.tr/en-US"}, { "id":"Suez", "title":"Suez Canal Container Terminal S.A.E.", "share": "20", "target": "8", "TEU": "5,000,000", "depth": "17.0", "url": "https://scct.com.eg/"}, { "id":"Euromax", "title":"Euromax Terminal Rotterdam B.V.", "share": "17.85", "target": "5", "TEU": "3,200,000", "depth": "17.65"}, { "id":"Antwerp", "title":"Antwerp Gateway NV", "share": "20", "target": "4", "TEU": "3,700,000", "depth": "16.0", "url": "http://www.dpworldantwerp.com/our-businesses/antwerp-gateway"}, { "id":"Seattle", "title":"SSA Terminals (Seattle), LLC", "share": "13.33", "target": "2", "TEU": "400,000", "depth": "15.2", "url": "http://www.ssamarine.com/locations/terminal-30/"}, { "id":"Busan", "title":"Busan Port Terminal Co., Ltd.", "share": "4.89", "target": "8", "TEU": "4,000,000", "depth": "15.0-16.0"}, { "id":"Abu", "title":"CSP Abu Dhabi Terminal L.L.C.", "share": "40", "target": "3", "TEU": "2,500,000 (Phase 2)", "depth": "18"/*, "remark":"Expected operation commencement date: third quarter in 2019"*/}, { "id":"Red", "title":"Red Sea Gateway Terminal", "share": "20", "target": "11", "TEU": "5,200,000", "depth": "18.0"}, { "id":"Chancay", "title":"COSCO SHIPPING Ports Chancay PERU S.A.<sup>*</sup>", "share": "60", "target": "2", "TEU": "1,000,000", "depth": "16-18", "target2": "2", "TEU2": "6,200,000", "depth2": "14", "remark":"* Under Construction"} ]; 

SyntaxError: invalid syntax (<ipython-input-24-299d5495bd3f>, line 1)